In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import time
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
import random

Using TensorFlow backend.


In [2]:
#Random start of population
#genomes as a list of gene sequences
def initialise():
    genomes =[]
    for i in range(8):
        genomes.append([np.random.randint(2, size=6)]) # add 5 randomly initialised indiviudals
    
    #entschachtelungsprozess
    ent = [5]*len(genomes)
    i = 0
    for k in genomes:
        for l in k:
            
            ent[i] =l
            i = i+1
    

    return ent

def craschcheck(popul):
    i=0
    for indi in popul:
        #print(indi)
        if check(indi) == True:
            print('Yey');
        if check(indi) == False:
            indi[2] = 1
            print(popul[i])
            popul[i] = indi
            print('Problem')
        
            
        i = i +1
                

            
def check(indi):
    for gen in indi:
        if gen == 1:
            return True
    return False


In [3]:
#usefull functions

def paring(mom,dad):
    return(np.concatenate((mom[:3], dad[3:]), axis=0))

def entschachteln(genos):
    #print(len(genos))
    neue = [0,0]
    i = 0
    for k in genos:
        for l in k:
            #print(l)
            neue[i] =l
            i = i+1

    return (neue)

def mutation(popul):
    position = random.randrange(0, len(popul))
    gencode = popul[position]
    
    decider = random.uniform(0, 1)
    posi = 0
    if decider < 0.8 :
       
        print('Mutation')
        posi = random.randrange(0, len(gencode))
    
        if gencode[posi] == 1:
            gencode[posi] = 0
        elif gencode[posi] == 0:
            gencode[posi] = 1
            
    popul[position] = np.array(gencode)
    
    return popul

def mutationeach(popul):
  
    i = 0    
    for gencode in popul:
        decider = random.uniform(0, 1)
        if decider < 0.351: 
            print('Muation')
            posi = 0
            posi = random.randrange(0, len(gencode))
           
            if gencode[posi] == 1:
                gencode[posi] = 0
            elif gencode[posi] == 0:
                gencode[posi] = 1
            
            popul[i] = np.array(gencode)
        
            
        i = i +1
    

            
    
    return popul
    

''' A one to one replacement reproduction '''
def selected(maxis, actualpop): 
    childs= []
    maxis= np.array(maxis)
    indices = maxis.argsort()[-5:][::-1]
    #print('Highest acc at')
    #print(indices)
    parents= []
    
   
    for ind in indices:    
        parents.append(actualpop[ind])
    
    random.shuffle(parents)
    f= 0
   
    while f < len(parents)-1: 
        mom = parents[f]
        f= f +1
        dad = parents[f]     
        child = paring(mom,dad)
        childs.append(child)
        child = paring(dad,mom)
        childs.append(child)
        
        
    return childs

In [4]:
def netzdurchlauf(acutalpop):
    accuracy_of_population= []
    
    print('Aktuelle Population')
    print(actualpop)
    for element in actualpop:
        batch_size = 128
        num_classes = 10
        epochs = 12

        # input image dimensions
        img_rows, img_cols = 28, 28

        # the data, split between train and test sets
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        if K.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, img_rows, img_cols)
        else:
            x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
            input_shape = (img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255
#             print('x_train shape:', x_train.shape)
#             print(x_train.shape[0], 'train samples')
#             print(x_test.shape[0], 'test samples')

        # convert class vectors to binary class matrices
        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)

        model = Sequential()




        model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
            ########
            
            

        for gen in element:
            if (gen== 0):
                model.add(Conv2D(64, (3, 3), activation='relu'))
            elif (gen == 1):
                model.add(Dense(128, activation='relu'))




        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss=keras.losses.categorical_crossentropy,
                          optimizer=keras.optimizers.Adadelta(),
                          metrics=['accuracy'])

        model.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_data=(x_test, y_test))
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        accuracy_of_population.append(score[1])
        
    return (accuracy_of_population)

In [5]:
def fortpflanzung (maxis, actualpop):
    childs= []
    maxis= np.array(maxis)
    indices = maxis.argsort()[-4:][::-1] # hier verändern, wenn wir individuen anzahl erhöhen die erste zahl muss die hälfte der anzahl sein
    #print('Highest acc at')
    #print(indices)
    parents= []
    
    
    for ind in indices:    
        parents.append(actualpop[ind])
   
    

    random.shuffle(parents)
    f= 0
 
    
    while f < len(parents): 
        #print(f)
        f = f -1
        mom = parents[f]
        f= f +1
        #print(f)
        dad = parents[f]    
        f= f +1
        decider = random.uniform(0, 1) # decides random how the cross-over works
        if decider < 0.5 :
            child = paring(mom,dad)
            childs.append(child)
        else:
            child = paring(dad,mom)
            childs.append(child)

    

    parents = np.concatenate((parents, childs), axis=0) 
    
 
    return parents

In [6]:

# Hauptmethode
start = time.time()


DURCHGÄNGE = 20 # variable to clarify number of generations
actualpop= initialise()

#actualpop= entschachteln(actualpop)
evalaccuris = []
saved = actualpop
i = 1

    
while i < DURCHGÄNGE+1:
    
    nummer = i
    accuri = netzdurchlauf(actualpop)
    print(' Accuracy of populationnummer: ')
    print( nummer)
    print(accuri)
    print('highest accuracy of populationnummer: ')
    print( nummer)
    print(np.amax(accuri))
    #print(actualpop)
    actualpop = fortpflanzung(accuri, actualpop)
    actualpop = mutationeach(actualpop)
    craschcheck(actualpop)
   
    evalaccuris.append([i , sum(accuri)/len(accuri),np.amax(accuri),actualpop[np.argmax(accuri)]])
    i = i+1
    
    
    



print('Wartezeit: ')
end = time.time()
seconds = end - start
minutes = seconds / 60
print(minutes)
hours = minutes/60
print(hours)

Aktuelle Population
[array([1, 0, 0, 1, 1, 1]), array([1, 1, 0, 1, 0, 0]), array([0, 0, 1, 1, 1, 1]), array([1, 0, 0, 0, 1, 1]), array([0, 1, 1, 0, 1, 0]), array([1, 1, 0, 1, 1, 0]), array([0, 1, 0, 1, 1, 0]), array([0, 1, 1, 1, 0, 0])]


W0930 09:27:21.608330 13676 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 09:27:21.635266 13676 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 09:27:21.645232 13676 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0930 09:27:21.679143 13676 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0930 09:27:21.681137 13676 deprecation_wrapper.py:119] From C:\Users\vivia\Ana

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 18s 303us/step - loss: 0.2720 - acc: 0.9161 - val_loss: 0.0641 - val_acc: 0.9780
Epoch 2/12
60000/60000 [==============================] - 12s 192us/step - loss: 0.0745 - acc: 0.9776 - val_loss: 0.0540 - val_acc: 0.9862
Epoch 3/12
60000/60000 [==============================] - 11s 187us/step - loss: 0.0530 - acc: 0.9840 - val_loss: 0.0364 - val_acc: 0.9875
Epoch 4/12
60000/60000 [==============================] - 11s 191us/step - loss: 0.0421 - acc: 0.9868 - val_loss: 0.0300 - val_acc: 0.9899
Epoch 5/12
60000/60000 [==============================] - 12s 196us/step - loss: 0.0352 - acc: 0.9890 - val_loss: 0.0359 - val_acc: 0.9883
Epoch 6/12
60000/60000 [==============================] - 12s 203us/step - loss: 0.0301 - acc: 0.9902 - val_loss: 0.0228 - val_acc: 0.9923
Epoch 7/12
60000/60000 [==============================] - 12s 198us/step - loss: 0.0279 - acc: 0.9917 - val_loss: 0.

60000/60000 [==============================] - 13s 224us/step - loss: 0.3086 - acc: 0.9014 - val_loss: 0.0741 - val_acc: 0.9759
Epoch 2/12
60000/60000 [==============================] - 13s 217us/step - loss: 0.0778 - acc: 0.9755 - val_loss: 0.0403 - val_acc: 0.9874
Epoch 3/12
60000/60000 [==============================] - 13s 217us/step - loss: 0.0541 - acc: 0.9835 - val_loss: 0.0327 - val_acc: 0.9898
Epoch 4/12
60000/60000 [==============================] - 13s 215us/step - loss: 0.0423 - acc: 0.9868 - val_loss: 0.0318 - val_acc: 0.9892
Epoch 5/12
60000/60000 [==============================] - 13s 209us/step - loss: 0.0356 - acc: 0.9887 - val_loss: 0.0220 - val_acc: 0.9929
Epoch 6/12
60000/60000 [==============================] - 12s 208us/step - loss: 0.0304 - acc: 0.9905 - val_loss: 0.0229 - val_acc: 0.9920
Epoch 7/12
60000/60000 [==============================] - 12s 203us/step - loss: 0.0268 - acc: 0.9914 - val_loss: 0.0221 - val_acc: 0.9928
Epoch 8/12
60000/60000 [==============

Epoch 9/12
60000/60000 [==============================] - 13s 224us/step - loss: 0.0286 - acc: 0.9907 - val_loss: 0.0330 - val_acc: 0.9895
Epoch 10/12
60000/60000 [==============================] - 14s 230us/step - loss: 0.0265 - acc: 0.9916 - val_loss: 0.0279 - val_acc: 0.9911
Epoch 11/12
60000/60000 [==============================] - 14s 227us/step - loss: 0.0239 - acc: 0.9925 - val_loss: 0.0293 - val_acc: 0.9905
Epoch 12/12
60000/60000 [==============================] - 14s 229us/step - loss: 0.0223 - acc: 0.9930 - val_loss: 0.0266 - val_acc: 0.9916
Test loss: 0.026569804297443624
Test accuracy: 0.9916
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 13s 218us/step - loss: 0.3578 - acc: 0.8843 - val_loss: 0.0722 - val_acc: 0.9750
Epoch 2/12
60000/60000 [==============================] - 11s 191us/step - loss: 0.0798 - acc: 0.9762 - val_loss: 0.0434 - val_acc: 0.9851
Epoch 3/12
60000/60000 [==============================] - 1

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



60000/60000 [==============================] - 11s 182us/step - loss: 0.0156 - acc: 0.9955 - val_loss: 0.0186 - val_acc: 0.9943
Test loss: 0.01855826778993469
Test accuracy: 0.9943
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 22s 364us/step - loss: 0.4178 - acc: 0.8657 - val_loss: 0.0553 - val_acc: 0.9824
Epoch 2/12
60000/60000 [==============================] - 11s 176us/step - loss: 0.0820 - acc: 0.9760 - val_loss: 0.0473 - val_acc: 0.9864
Epoch 3/12
60000/60000 [==============================] - 11s 176us/step - loss: 0.0552 - acc: 0.9841 - val_loss: 0.0407 - val_acc: 0.9890
Epoch 4/12
60000/60000 [==============================] - 11s 176us/step - loss: 0.0395 - acc: 0.9884 - val_loss: 0.0281 - val_acc: 0.9903
Epoch 5/12
60000/60000 [==============================] - 11s 176us/step - loss: 0.0319 - acc: 0.9900 - val_loss: 0.0389 - val_acc: 0.9880
Epoch 6/12
60000/60000 [==============================] - 10s 174us/step -

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



60000/60000 [==============================] - 15s 255us/step - loss: 0.0335 - acc: 0.9901 - val_loss: 0.0235 - val_acc: 0.9932
Epoch 6/12
60000/60000 [==============================] - 15s 249us/step - loss: 0.0286 - acc: 0.9917 - val_loss: 0.0238 - val_acc: 0.9932
Epoch 7/12
60000/60000 [==============================] - 16s 262us/step - loss: 0.0242 - acc: 0.9926 - val_loss: 0.0197 - val_acc: 0.9944
Epoch 8/12
60000/60000 [==============================] - 2043s 34ms/step - loss: 0.0215 - acc: 0.9935 - val_loss: 0.0231 - val_acc: 0.9935
Epoch 9/12
60000/60000 [==============================] - 10s 170us/step - loss: 0.0201 - acc: 0.9940 - val_loss: 0.0194 - val_acc: 0.9938
Epoch 10/12
60000/60000 [==============================] - 10s 171us/step - loss: 0.0167 - acc: 0.9949 - val_loss: 0.0322 - val_acc: 0.9908
Epoch 11/12
60000/60000 [==============================] - 10s 168us/step - loss: 0.0164 - acc: 0.9952 - val_loss: 0.0191 - val_acc: 0.9936
Epoch 12/12
60000/60000 [==========

KeyboardInterrupt: 

In [ ]:
#Visualisation
df = pd.DataFrame(data =evalaccuris)
df.rename(columns={0: 'Generation',1: 'Accuracy Mean',2:'Accuracy Best',3: 'Best Individum'}, inplace=True)
df
fr =sns.regplot(x=df['Generation'],y=df['Accuracy Best'])
plt.ylim(0.990, 1)

plt.figure(figsize=(25,25))
plt.tight_layout()
plt.show()
fig = fr.get_figure()
fig.savefig('Best_Accuracies_in_Generation ( M: 0.15).png')

In [ ]:
#Visualisation von dem durchschnitt der accuracy

fr =sns.regplot(x=df['Generation'],y=df['Accuracy Mean'])

plt.ylim(0.990, 1)

fig = fr.get_figure()
fig.savefig('Average_Accuracy_in_Generation ( M: 0.15).png')


In [ ]:

 
ax1 =sns.regplot(x=df['Generation'],y=df['Accuracy Best'])
ax0 =sns.regplot(x=df['Generation'],y=df['Accuracy Mean'])
fig = fr.get_figure()
fig.savefig('Accuracies Mutation: 3.35.png', bbox_inches='tight')

In [ ]:
# wievielschwnakung 


In [ ]:
rettung=evalaccuris

In [ ]:
print(evalaccuris)

In [ ]:

print(actualpop)

In [ ]:
def craschcheck(popul):
    i=0
    for indi in popul:
        #print(indi)
        if check(indi) == True:
            print('Yey');
        if check(indi) == False:
            indi[2] = 1
            print(popul[i])
            popul[i] = indi
            print('Problem')
        
            
        i = i +1
                

            
def check(indi):
    for gen in indi:
        if gen == 1:
            return True
    return False



In [ ]:
la = [[0, 0, 0, 0, 0, 0],[0,0,0,1,0,0],[0,0,0,0,0,0]]

print (la)

craschcheck(la)
print(la)